<a href="https://colab.research.google.com/github/pisceno/AI-Generative/blob/main/clasificacion_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os
import numpy as np
import csv

# ====================================================================
# CONFIGURACIÓN DE DESCARGA
# ====================================================================
# URL RAW DE TU REPOSITORIO DE GITHUB
# Esta URL debe funcionar porque apunta al servidor de contenido crudo.
RAW_URL_WDBC = "https://raw.githubusercontent.com/pisceno/data_sets/main/wdbc.data"
# Nombre del archivo que se guardará localmente
DATA_PATH_LOCAL = "wdbc.data"

print(f"Descargando {DATA_PATH_LOCAL} desde GitHub...")

try:
    response = requests.get(RAW_URL_WDBC)
    response.raise_for_status() # Lanza un error si la descarga falla

    # Guarda el contenido en un archivo local llamado 'wdbc.data'
    with open(DATA_PATH_LOCAL, 'wb') as f:
        f.write(response.content)

    print(f"✅ Archivo {DATA_PATH_LOCAL} descargado y guardado localmente.")

except requests.exceptions.HTTPError as errh:
    print(f"❌ Error HTTP al descargar. Verifica que el archivo exista en la URL RAW.")
except Exception as e:
    print(f"❌ Ocurrió un error general en la descarga: {e}")

Descargando wdbc.data desde GitHub...
✅ Archivo wdbc.data descargado y guardado localmente.


In [ ]:
%%writefile clasificacion_NN.py
import numpy as np
import csv
import sys
import os

# ==============================================================================
# CONFIGURACIÓN (Ruta local)
# ==============================================================================
# ¡SOLO EL NOMBRE DEL ARCHIVO LOCAL! El script lo buscará en la máquina de Colab.
DATA_PATH = "wdbc.data"
LEARNING_RATE = 0.01
ITERATIONS = 50000

# ==============================================================================
# FUNCIONES NÚCLEO (IMPLEMENTACIÓN DESDE CERO)
# ==============================================================================

def sigmoid(z):
    """Función de activación Sigmoid: sigma(z) = 1 / (1 + e^(-z))"""
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, w):
    """Cálculo del Error (Costo) - Binary Cross-Entropy (Log Loss)"""
    m = len(y)
    h = sigmoid(X @ w)
    h = np.clip(h, 1e-15, 1 - 1e-15)
    cost = (-1/m) * (y.T @ np.log(h) + (1 - y).T @ np.log(1 - h))
    return cost

def gradient_descent(X, y, w, learning_rate, iterations):
    """Algoritmo de optimización de Gradiente Descendente."""
    m = len(y)
    for i in range(iterations):
        h = sigmoid(X @ w)
        gradient = (1/m) * X.T @ (h - y)
        w -= learning_rate * gradient
    return w

def standardize_features(X):
    """Estandariza las características (media 0, desviación estándar 1)."""
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    sigma[sigma == 0] = 1
    X_norm = (X - mu) / sigma
    return X_norm

# ==============================================================================
# EJECUCIÓN PRINCIPAL
# ==============================================================================
if __name__ == "__main__":

    # Verificación de existencia de archivo local
    if not os.path.exists(DATA_PATH):
        print(f"Error: No se encontró el archivo de datos '{DATA_PATH}'.")
        print("Asegúrate de ejecutar la celda de descarga de GitHub previamente.")
        sys.exit(1)

    try:
        # 1. CARGAR Y PREPROCESAR LOS DATOS
        data = []
        # 'open(DATA_PATH, 'r')' busca el archivo localmente
        with open(DATA_PATH, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                # Ignorar ID (columna 0) y tomar el resto
                data.append(row[1:])

        data = np.array(data)

        # Separar, codificar y estandarizar
        y_raw = data[:, 0]
        X_raw = data[:, 1:].astype(float)
        y = np.where(y_raw == 'M', 1, 0) # M=1, B=0

        X_norm = standardize_features(X_raw)
        X = np.hstack([np.ones((X_norm.shape[0], 1)), X_norm]) # Añadir Bias (w0)

        # 2. ENTRENAMIENTO DEL MODELO
        num_features = X.shape[1]
        initial_w = np.zeros(num_features)
        final_w = gradient_descent(X, y, initial_w, LEARNING_RATE, ITERATIONS)

        # 3. CÁLCULO DEL ERROR FINAL
        final_error = compute_cost(X, y, final_w)

        # 4. GENERAR LA SALIDA REQUERIDA (w0 w1 ... wk E)
        output = []
        output.append(f"w0: {final_w[0]:.6f}")
        for i in range(1, num_features):
            output.append(f"w{i}: {final_w[i]:.6f}")
        output.append(f"E: {final_error:.6f}")

        print(' '.join(output))

    except Exception as e:
        print(f"Ocurrió un error durante la ejecución: {e}")

Writing clasificacion_NN.py


In [3]:
!python clasificacion_NN.py

python3: can't open file '/content/clasificacion_NN.py': [Errno 2] No such file or directory


In [ ]:
from google.colab import files

# Esto abrirá una ventana para que selecciones el archivo en tu computadora
print("Por favor, sube el archivo wdbc.data")
uploaded = files.upload()

Por favor, sube el archivo wdbc.data


Saving wdbc.data to wdbc (1).data


In [ ]:
import os

# 1. Renombrar el archivo subido (el nombre que Colab le dio) a 'wdbc.data'
try:
    os.rename("wdbc (1).data", "wdbc.data")
    print("✅ Archivo renombrado exitosamente a wdbc.data.")
except FileNotFoundError:
    # Si 'wdbc (1).data' no existe, puede ser que el nombre original sí
    if os.path.exists("wdbc.data"):
        print("El archivo wdbc.data ya existe. Listo para el Paso 2.")
    else:
        print("❌ Error: Los archivos no se encuentran en el entorno. Vuelve a ejecutar la celda de subida.")

✅ Archivo renombrado exitosamente a wdbc.data.


In [6]:
%%writefile clasificacion_NN.py
import numpy as np
import csv
import sys
import os

# ==============================================================================
# CONFIGURACIÓN
# ==============================================================================
DATA_PATH = "wdbc.data"
LEARNING_RATE = 0.01
ITERATIONS = 50000

# ==============================================================================
# FUNCIONES NÚCLEO
# ==============================================================================

def sigmoid(z):
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, w):
    m = len(y)
    h = sigmoid(X @ w)
    h = np.clip(h, 1e-15, 1 - 1e-15)
    cost = (-1/m) * (y.T @ np.log(h) + (1 - y).T @ np.log(1 - h))
    return cost

def gradient_descent(X, y, w, learning_rate, iterations):
    m = len(y)
    for i in range(iterations):
        h = sigmoid(X @ w)
        gradient = (1/m) * X.T @ (h - y)
        w -= learning_rate * gradient
    return w

def standardize_features(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    sigma[sigma == 0] = 1
    X_norm = (X - mu) / sigma
    return X_norm

# ==============================================================================
# EJECUCIÓN PRINCIPAL
# ==============================================================================
if __name__ == "__main__":

    if not os.path.exists(DATA_PATH):
        print(f"Error: No se encontró el archivo de datos '{DATA_PATH}'.")
        sys.exit(1)

    try:
        data = []
        with open(DATA_PATH, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                data.append(row[1:])

        data = np.array(data)

        y_raw = data[:, 0]
        X_raw = data[:, 1:].astype(float)
        y = np.where(y_raw == 'M', 1, 0)

        X_norm = standardize_features(X_raw)
        X = np.hstack([np.ones((X_norm.shape[0], 1)), X_norm])

        num_features = X.shape[1]
        initial_w = np.zeros(num_features)
        final_w = gradient_descent(X, y, initial_w, LEARNING_RATE, ITERATIONS)

        final_error = compute_cost(X, y, final_w)

        output = []
        output.append(f"w0: {final_w[0]:.6f}")
        for i in range(1, num_features):
            output.append(f"w{i}: {final_w[i]:.6f}")
        output.append(f"E: {final_error:.6f}")

        print(' '.join(output))

    except Exception as e:
        print(f"Ocurrió un error durante la ejecución: {e}")

Writing clasificacion_NN.py


In [7]:
!python clasificacion_NN.py

Error: No se encontró el archivo de datos 'wdbc.data'.
